In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns

In [97]:
#df = pd.read_csv('Data/births_toy.csv', index_col=[0])
df = pd.read_csv('Data/US_births(2018).csv')
col_dict = pd.read_excel('Data/Column_dictionary.xlsx', header=None)

/rds/general/user/cam119/home/anaconda3/envs/ML_project/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [98]:
df.shape

(3801534, 55)

In [99]:
df.head()

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,...,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,...,0,2,0,188,4,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,...,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,...,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,...,0,1,4,220,3,1,N,0,M,0


In [100]:
col_dict.head()

,0,1
0,ATTEND,birth_attendant
1,BFACIL,birth_place
2,BMI,bmi
3,CIG_0,cigs_before_preg
4,DBWT,birthweight_g


In [101]:
df.columns = col_dict[1]
df.columns = map(str.lower, df.columns)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3801534 entries, 0 to 3801533
Data columns (total 55 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   birth_attendant        int64  
 1   birth_place            int64  
 2   bmi                    float64
 3   cigs_before_preg       int64  
 4   birthweight_g          int64  
 5   last_norm_menses_mn    int64  
 6   last_norm_menses_yr    int64  
 7   marital_stat           object 
 8   birth_mn               int64  
 9   birth_time             int64  
 10  birth_dy               int64  
 11  birth_yr               int64  
 12  m_deliveryweight       int64  
 13  f_age                  int64  
 14  f_education            int64  
 15  f_hispanic             int64  
 16  f_race15               int64  
 17  f_race31               int64  
 18  f_race6                int64  
 19  last_live_birth_mn     int64  
 20  last_other_preg_mn     int64  
 21  last_preg_mn           int64  
 22  sex_imp           

* Cleaning data: removing useless columns, coding NAs, coding in binary, and removing entries with impossible values (high/low birthweight)

In [103]:
df = df.drop(['last_other_preg_mn', 
              'sex_imp', 
              'm_age_imp', 
              'm_marital_stat_imp', 
              'm_race_imp',
              'payment_det'], axis=1)

In [104]:
def replace_na(data, column_name, value):
    data[column_name] = data[column_name].mask(data[column_name] == value, np.nan)

In [105]:
replace_na(df, 'bmi', 99.9)
replace_na(df, 'm_nativity', 3)

In [106]:
columns_9 = ['birth_attendant','birth_place','f_education','f_hispanic','f_race6','m_education','m_hispanic',
             'infections','m_morbidity','riskf','delivery_method','payment']
              
for i in columns_9:
    replace_na(df, i, 9)

In [107]:
columns_99 = ['cigs_before_preg','last_norm_menses_mn','f_age','f_race15','f_race31','m_height_in',
             'mn_prenatalcare_began','num_prenatal_visits','prior_births_dead','prior_births_living',
             'prior_terminations','num_prev_cesareans','weight_gain']

for i in columns_99:
    replace_na(df, i, 99)

In [108]:
columns_888 = ['last_preg_mn','last_live_birth_mn']

for i in columns_888:
    replace_na(df, i, 888)

In [109]:
columns_999 = ['last_live_birth_mn','last_preg_mn','m_deliveryweight','prepreg_weight']

for i in columns_999:
    replace_na(df, i, 999)

In [110]:
columns_9999 = ['birthweight_g','last_norm_menses_yr','birth_time']

for i in columns_9999:
    replace_na(df, i, 9999)

In [111]:
columns_U = ['gonorrhea','labour_induced','admit_icu','m_transferred','prev_cesarean']

for i in columns_U:
    replace_na(df, i, 'U')

In [112]:
df['infections'] = df['infections'].mask(df['infections'] == 1, 'N')
df['infections'] = df['infections'].mask(df['infections'] == 0, 'Y')

df['m_morbidity'] = df['m_morbidity'].mask(df['m_morbidity'] == 1, 'N')
df['m_morbidity'] = df['m_morbidity'].mask(df['m_morbidity'] == 0, 'Y')

df['riskf'] = df['riskf'].mask(df['riskf'] == 1, 'N')
df['riskf'] = df['riskf'].mask(df['riskf'] == 0, 'Y')

In [113]:
df.describe()

,birth_attendant,birth_place,bmi,cigs_before_preg,birthweight_g,last_norm_menses_mn,last_norm_menses_yr,birth_mn,birth_time,birth_dy,...,mn_prenatalcare_began,num_prenatal_visits,prior_births_dead,prior_births_living,prior_terminations,prepreg_weight,delivery_method,res_status,num_prev_cesareans,weight_gain
count,3.799026e+06,3.801262e+06,3.715334e+06,3.783364e+06,3.798574e+06,3.623781e+06,3.634686e+06,3.801534e+06,3.801218e+06,3.801534e+06,...,3.705862e+06,3.705917e+06,3.791288e+06,3.793816e+06,3.788758e+06,3.725000e+06,3.799906e+06,3.801534e+06,3.797480e+06,3.684346e+06
mean,1.326893e+00,1.035268e+00,2.716780e+01,1.105816e+00,3.261649e+03,6.608613e+00,2.017248e+03,6.560880e+00,1.228447e+03,4.059289e+00,...,2.901963e+00,1.129048e+01,1.514156e-02,1.138918e+00,4.299018e-01,1.590928e+02,1.796411e+00,1.331968e+00,2.225126e-01,2.947331e+01
std,7.482103e-01,3.153267e-01,6.759659e+00,4.743047e+00,5.904779e+02,3.477206e+00,4.334201e-01,3.419287e+00,6.315904e+02,1.852540e+00,...,1.522518e+00,4.200561e+00,1.820914e-01,1.304771e+00,8.964494e-01,4.191389e+01,1.162220e+00,5.339529e-01,5.898300e-01,1.515072e+01
min,1.000000e+00,1.000000e+00,1.300000e+01,0.000000e+00,2.270000e+02,1.000000e+00,2.016000e+03,1.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.500000e+01,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,2.230000e+01,0.000000e+00,2.960000e+03,4.000000e+00,2.017000e+03,4.000000e+00,7.590000e+02,3.000000e+00,...,2.000000e+00,9.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.300000e+02,1.000000e+00,1.000000e+00,0.000000e+00,2.000000e+01
50%,1.000000e+00,1.000000e+00,2.570000e+01,0.000000e+00,3.300000e+03,7.000000e+00,2.017000e+03,7.000000e+00,1.236000e+03,4.000000e+00,...,3.000000e+00,1.200000e+01,0.000000e+00,1.000000e+00,0.000000e+00,1.500000e+02,1.000000e+00,1.000000e+00,0.000000e+00,2.900000e+01
75%,1.000000e+00,1.000000e+00,3.070000e+01,0.000000e+00,3.629000e+03,1.000000e+01,2.017000e+03,9.000000e+00,1.733000e+03,6.000000e+00,...,3.000000e+00,1.300000e+01,0.000000e+00,2.000000e+00,1.000000e+00,1.800000e+02,3.000000e+00,2.000000e+00,0.000000e+00,3.800000e+01
max,5.000000e+00,7.000000e+00,6.990000e+01,9.800000e+01,8.165000e+03,1.200000e+01,2.018000e+03,1.200000e+01,2.359000e+03,7.000000e+00,...,1.000000e+01,9.800000e+01,1.300000e+01,1.800000e+01,2.800000e+01,3.750000e+02,6.000000e+00,4.000000e+00,1.100000e+01,9.800000e+01


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3801534 entries, 0 to 3801533
Data columns (total 39 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   birth_attendant        float64
 1   birth_place            float64
 2   bmi                    float64
 3   cigs_before_preg       float64
 4   birthweight_g          float64
 5   birth_mn               int64  
 6   birth_time             float64
 7   birth_dy               int64  
 8   m_deliveryweight       float64
 9   f_age                  float64
 10  f_education            float64
 11  f_hispanic             float64
 12  f_race6                float64
 13  gonorrhea              object 
 14  labour_induced         object 
 15  m_age                  int64  
 16  m_nativity             float64
 17  m_education            float64
 18  m_hispanic             float64
 19  admit_icu              object 
 20  m_race6                int64  
 21  m_transferred          object 
 22  m_height_in       

In [115]:
def convert_binary(data, col_name):
    data[col_name] = data[col_name].mask(data[col_name] == 'Y', 1)
    data[col_name] = data[col_name].mask(data[col_name] == 'N', 0)

cols = ['gonorrhea','labour_induced','admit_icu','m_transferred','infections','m_morbidity','prev_cesarean', 'riskf']

for i in cols:
    convert_binary(df, i)

In [116]:
df['infant_sex'] = df['infant_sex'].mask(df['infant_sex'] == 'M', 1)
df['infant_sex'] = df['infant_sex'].mask(df['infant_sex'] == 'F', 0)

In [117]:
#def convert_int(data, col_name):
 #   data[col_name] = data[col_name].astype(int)
    
#cols = ['gonorrhea','labour_induced','admit_icu','m_transferred','infections','m_morbidity','riskf','infant_sex']

#for i in cols:
 #   convert_int(df, i)

In [118]:
#drop recoded
df = df.drop(['f_race15','f_race31','m_race15','m_race31'], axis=1)

In [119]:
#drop those with too many missing values
df = df.drop(['last_live_birth_mn','last_preg_mn'], axis=1)

In [120]:
print('{:.2f}% of infants born in US in 2018 were LBW infants'.\
      format(1e2*len(df[df['birthweight_g'] < 2500])/len(df)))
print('{:.2f}% of infants born in US in 2018 were very-LBW infants'.\
      format(1e2*len(df[df['birthweight_g'] < 1500])/len(df)))

8.27% of infants born in US in 2018 were LBW infants
1.37% of infants born in US in 2018 were very-LBW infants


In [121]:
#idx = df[df['birthweight_g'] < 300].index
#df.drop(idx, inplace=True)

In [122]:
#idx = df[df['birthweight_g'] > 7000].index
#df.drop(idx, inplace=True)

In [123]:
#dropping marital_stat because something wrong with encoding
#dropping weight_gain because will encode weight_change later instead
df = df.drop(['marital_stat','weight_gain'], axis=1)

* Encoding time between birth and menses, instead of date of last menses

In [124]:
df['last_menses_day'] = 15
   
last_menses = pd.DataFrame({'year': df['last_norm_menses_yr'], 
                            'month': df['last_norm_menses_mn'], 
                            'day': df['last_menses_day']}) 
df['last_menses'] = pd.to_datetime(last_menses)

df = df.drop(['last_norm_menses_yr','last_norm_menses_mn','last_menses_day'], axis=1)

In [125]:
dob = pd.DataFrame({'year': df['birth_yr'],
                    'month': df['birth_mn'], 
                    'day': df['birth_dy']})  
df['dob'] = pd.to_datetime(dob)

df = df.drop(['birth_yr'], axis=1)

In [126]:
df['time_since_menses'] = (df['dob'] - df['last_menses'])

df = df.drop(['last_menses','dob'], axis=1)

In [129]:
df['time_since_menses']

0          262.0
1            NaN
2          262.0
3          232.0
4            NaN
           ...  
3801529    231.0
3801530    262.0
3801531    232.0
3801532    140.0
3801533    265.0
Name: time_since_menses, Length: 3801534, dtype: float64

In [128]:
df['time_since_menses'] = df['time_since_menses'].dt.days

In [ ]:
df.to_csv('Data/Processed_data_full.csv')
#df.to_csv('Data/Processed_data.csv')

#### Imputation

* Tried making a processing pipeline, for both numeric and categorical data - currently not working 

In [28]:
#from sklearn.pipeline import Pipeline
#from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [143]:
#df_numeric = df.select_dtypes(include = ['number'])
#numeric_cols = df_numeric.columns

#df_cat = df.select_dtypes(include = ['object'])
#cat_cols = df_cat.columns

In [144]:
#numeric_transformer = Pipeline(steps=[
  #  ('imputer', IterativeImputer(random_state=0))])
#could add ('scaler', StandardScaler())

#categorical_transformer = Pipeline(steps=[
 #   ('imputer', SimpleImputer(strategy='most_frequent'))])
#could add ('onehot', OneHotEncoder())

##Or could I use np.round(pd.DataFrame(imputer.fit_transform(df))) for the categorical?

#preprocessor = ColumnTransformer(
 #   transformers=[
  #      ('num', numeric_transformer, numeric_cols),
   #     ('cat', categorical_transformer, cat_cols)])

In [145]:
#df_processed = preprocessor.fit_transform(df)

In [146]:
#df_processed = pd.DataFrame(df_processed)
#df_colnames = df.columns
#df_processed.columns = df_colnames

In [ ]:
#df_processed.head()

* Iterative imputer

In [ ]:
imputer = IterativeImputer(random_state=0)
df_imputed = pd.DataFrame(imputer.fit_transform(df))

In [ ]:
df_colnames = df.columns
df_imputed.columns = df_colnames

In [121]:
df_imputed.to_csv('Data/MICE_data_toy.csv')

* Post-imputation final processing

In [ ]:
df_imputed = pd.read_csv('Data/MICE_data2.csv')

In [ ]:
df_imputed['weight_change'] = df_imputed['m_deliveryweight'] - df_imputed['prepreg_weight']

In [ ]:
df_bin = df_imputed.copy()
df_bin['birthweight_bin'] = np.where(df_bin['birthweight_g'] < 2500, 1, 0)

df_cat = df_bin.copy()
df_cat['birthweight_cat'] = np.where(df_cat['birthweight_g'] < 1500, 2, df_cat['birthweight_bin'])

df_bin = df_bin.drop(['birthweight_g'], axis=1)
df_cat = df_cat.drop(['birthweight_g', 'birthweight_bin'], axis=1)

In [ ]:
df_imputed.to_csv('Data/MICE_processed.csv')
df_bin.to_csv('Data/MICE_bin_processed.csv')
df_cat.to_csv('Data/MICE_cat_processed.csv')

* Trying KNN imputation to see if this is better for categorical numerical variables 

In [10]:
df = pd.read_csv('Data/Processed_data_full.csv')

/rds/general/user/cam119/home/anaconda3/envs/ML_project/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df.head()

,Unnamed: 0,birth_attendant,birth_place,bmi,cigs_before_preg,birthweight_g,last_norm_menses_mn,last_norm_menses_yr,marital_stat,birth_mn,...,prior_births_dead,prior_births_living,prior_terminations,prepreg_weight,delivery_method,res_status,prev_cesarean,num_prev_cesareans,infant_sex,weight_gain
0,0,1.0,1.0,30.7,0.0,3657.0,4.0,2017.0,1,1,...,0.0,1.0,2.0,190.0,1.0,2,0.0,0.0,1,41.0
1,1,1.0,1.0,33.3,2.0,3242.0,NaN,NaN,2,1,...,0.0,2.0,0.0,188.0,4.0,2,1.0,2.0,0,0.0
2,2,1.0,1.0,30.0,0.0,3470.0,4.0,2017.0,1,1,...,0.0,1.0,0.0,215.0,1.0,1,0.0,0.0,1,58.0
3,3,3.0,1.0,23.7,0.0,3140.0,5.0,2017.0,2,1,...,0.0,2.0,0.0,138.0,1.0,2,0.0,0.0,0,0.0
4,4,1.0,1.0,35.5,0.0,2125.0,NaN,NaN,1,1,...,0.0,1.0,4.0,220.0,3.0,1,0.0,0.0,1,0.0


In [107]:
df_KNN = df.copy()

In [109]:
KNN_imputer = KNNImputer(n_neighbors=5)
df_KNN = KNN_imputer.fit_transform(df)
df_KNN = pd.DataFrame(df_KNN)

In [113]:
df_colnames = df.columns
df_KNN.columns = df_colnames

In [122]:
df_knn_bin = pd.DataFrame.copy(df_KNN)
df_knn_bin['birthweight_bin'] = np.where(df_knn_bin['birthweight_g'] < 2500, 1, 0)

In [123]:
df_knn_cat = pd.DataFrame.copy(df_knn_bin)
df_knn_cat['birthweight_cat'] = np.where(df_knn_cat['birthweight_g'] < 1500, 2, df_knn_cat['birthweight_bin'])

In [124]:
df_knn_bin = df_knn_bin.drop(['birthweight_g'], axis=1)
df_knn_cat = df_knn_cat.drop(['birthweight_g', 'birthweight_bin'], axis=1)

In [125]:
df_KNN.to_csv('Data/KNN_data_toy.csv')
df_knn_bin.to_csv('Data/KNN_data_bin_toy.csv')
df_knn_cat.to_csv('Data/KNN_data_cat_toy.csv')